In [3]:
import numpy as np
from qiskit.quantum_info import Pauli, random_clifford
import pickle
import os

In [4]:
def tensor_prod(*tensors):
    if len(tensors) == 2:
        return np.kron(tensors[0], tensors[1])
    else:
        return np.kron(tensors[0], tensor_prod(*tensors[1:]))
    
def hermitian(matrix):
    return np.allclose(matrix, matrix.conj().T)

def trace_one(matrix):
    return np.isclose(np.trace(matrix), 1)

def positive_semi_definite(matrix, tol=1e-8):
    return np.all(np.linalg.eigvals(matrix) + tol >= 0)

def is_legal(matrix):
    return hermitian(matrix) and trace_one(matrix) and positive_semi_definite(matrix)

def int_to_bin_list(n, length):
    bin_list = np.zeros(length)
    bin_list[n] = 1
    return bin_list

def single_sample(prob_list):
    assert np.isclose(sum(prob_list), 1), "probability does not sum up to 1"
    rd = np.random.random()
    inf, sup = 0, 0
    for i, e in enumerate(prob_list):
        sup += e
        if inf <= rd <= sup:
            return i
        else:
            inf = sup
    raise ValueError("random value does not meet any interval")

class QuantumState():
    def __init__(self, num_qubits:int, num_shots:int, batch_size:int, pauli_observables:list, veri:bool):
        self._num_qubits = num_qubits
        self._observables = pauli_observables
        self._batch_size = batch_size
        self._num_shots = num_shots
        self._veri = veri
        self._dm = None
        self._entangled = None
        
    @property
    def dm(self):
        return self._dm
    
    @dm.setter
    def dm(self, new_dm):
        if not (self._veri or is_legal(new_dm)):
            raise ValueError("density matrix is not physical")
        else:
            self._dm = new_dm
    
    def set_dm(self):
        raise NotImplementedError("without information to construct density matrix")
    
    def random_evolve(self):
        self._U = random_clifford(self._num_qubits).to_matrix()
        self._dm = self._U @ self.dm @ np.conj(self._U).T
    
    def single_shot_measure(self):
        prob_list = [self._dm[i, i] for i in range(2 ** self._num_qubits)]
        single_shot_state = int_to_bin_list(single_sample(prob_list), 2 ** self._num_qubits)
        del self._dm
        self._state = single_shot_state
    
    def reconstruct_dm(self):
        dim = 2 ** self._num_qubits
        return (dim + 1) * (np.conj(self._U).T @ np.outer(self._state, self._state) @ self._U) - np.eye(dim)

    # def classical_shadow(self):
    #     shadows = {obs: [] for obs in self._observables}
    #     temp_shadows = {obs: [] for obs in self._observables}
    #     dm_copy = self._dm
    #     for _ in range(self._num_shots // self._batch_size):
    #         for _ in range(self._batch_size):
    #             self._dm = dm_copy
    #             self.random_evolve()
    #             self.single_shot_measure()
    #             rdm = self.reconstruct_dm()
    #             for k, v in temp_shadows.items():
    #                 v.append(np.trace(Pauli(k).to_matrix() @ rdm))
    #         for k, v in shadows.items():
    #             v.append(np.mean(temp_shadows[k]))
    #         temp_shadows = {obs: [] for obs in self._observables}
    #     del temp_shadows
    #     return {k: np.median(v) for k, v in shadows.items()}
    
    def classical_shadow(self):
        shadows = {obs: [] for obs in self._observables}
        dm_copy = self._dm
        for _ in range(self._num_shots // self._batch_size):
            snapshots = []
            for _ in range(self._batch_size):
                self._dm = dm_copy
                self.random_evolve()
                self.single_shot_measure()
                snapshots.append(self.reconstruct_dm())
            mean = np.mean(np.stack(snapshots), axis=0)
            for k, v in shadows.items():
                v.append(np.trace(Pauli(k).to_matrix() @ mean))
        return {k: np.median(v) for k, v in shadows.items()}

In [8]:
def partial_transpose(matrix):
    if matrix.shape != (4, 4):
        raise ValueError("Input matrix must be 4x4.")
    result = np.zeros((4, 4), dtype=matrix.dtype)
    for i in range(2):
        for j in range(2):
            for k in range(2):
                for l in range(2):
                    result[2*i + k, 2*j + l] = matrix[2*i + k, 2*j + l]
                    result[2*i + k, 2*j + l] = matrix[2*i + l, 2*j + k]
    return result

In [26]:
import pandas as pd
from IPython.display import clear_output

data = []
count = 0
while count < 50000:
    dm = np.zeros((4, 4))
    for i in range(4):
        for j in range(i, 4):
            r1 = 10 * np.random.random() - 5
            r2 = 10 * np.random.random() - 5
            if j == i:
                dm[j, i] = r1
            else:
                dm[j, i] = r1 + j * r2
    dm = np.conj(dm).T @ dm / np.trace(np.conj(dm).T @ dm)
    label = not positive_semi_definite(partial_transpose(dm))
    if label is True:
        data.append({'matrix': dm, 'label':True})
        count += 1
        clear_output()
        print(f"entangled: {count} sample generated")
count = 0
while count < 50000:
    dm = np.zeros((4, 4))
    for i in range(4):
        for j in range(i, 4):
            r1 = 10 * np.random.random() - 5
            r2 = 10 * np.random.random() - 5
            if j == i:
                dm[j, i] = r1
            else:
                dm[j, i] = r1 + j * r2
    dm = np.conj(dm).T @ dm / np.trace(np.conj(dm).T @ dm)
    label = not positive_semi_definite(partial_transpose(dm))
    if label is False:
        data.append({'matrix': dm, 'label':False})
        count += 1
        clear_output()
        print(f"entangled: {count} sample generated")
clear_output()        
df = pd.DataFrame(data)
df = df.sample(frac=1).reset_index(drop=True)
# df.to_hdf('output/data.h5', key='df', mode='w')
# df_loaded = pd.read_hdf('data.h5', key='df')
df.to_csv('output/random4dim.csv')

In [27]:
df = pd.DataFrame(data)
df = df.sample(frac=1).reset_index(drop=True)
# df.to_hdf('output/data.h5', key='df', mode='w')
# df_loaded = pd.read_hdf('data.h5', key='df')
df.to_csv('output/random4dim.csv', index=False)